# Convolutional Neural Networsk Project

## Sub-project 1:

## Solve Fashion_MNIST with LeNet architecture

In this project you will implement the LeNet architecture of Convolutional Neural Networks. First you will download the Fashion-MNIST dataset. Split into train/validation/test datasets and train the network. Finally, plot the learning curves (train/validation loss and accuracy) and show the confusion matrix.

In [15]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor, transforms
from torch.utils.data import DataLoader
from torch import optim

from sklearn.model_selection import train_test_split

### 1. Download Fashion-MNIST

In [16]:
transform = transforms.Compose([
    transforms.Pad(2),           # pads 28x28 to 32x32
    transforms.ToTensor(),       # converts to PyTorch tensor
])

data = datasets.FashionMNIST(
    root = 'data/mnist',
    train = True,
    transform = transform,
    download = True,
)

test_data = datasets.FashionMNIST(
    root = 'data/mnist',
    train = False,
    transform = transform,
)

len(data)

60000

### 2. Split the data into train / validation / test subsets. Make mini-batches, if necesssary.

In [17]:
from torch.utils.data import DataLoader

train_set, val_set = torch.utils.data.random_split(data, [50000, 10000])

print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")

loaders = {
    'train' : torch.utils.data.DataLoader(train_set,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=1),

    'validation'  : torch.utils.data.DataLoader(val_set,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=1),
    'test'  : torch.utils.data.DataLoader(test_data,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=1),
}
print(f"Mini batch training set size: {len(loaders['train'])}")
print(f"Mini batch validation set size: {len(loaders['validation'])}")
print(f"Mini batch test set size: {len(loaders['test'])}")

Training set size: 50000
Validation set size: 10000
Mini batch training set size: 500
Mini batch validation set size: 100
Mini batch test set size: 100


### 3. Build the LeNet model

In [26]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 5),          # input: (1, 32, 32), output: (6, 28, 28)
            nn.Tanh(),
            nn.AvgPool2d(2, 2)           # output: (6, 14, 14)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),         # input: (6, 14, 14), output: (16, 10, 10)
            nn.Tanh(),
            nn.AvgPool2d(2, 2)           # output: (16, 5, 5)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 120, 5),        # input: (16, 5, 5), output: (120, 1, 1)
            nn.Tanh()
        )
        
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        return x
    
conv_nn = CNN()
print(conv_nn)

loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(conv_nn.parameters(), lr=0.001)
for parameter in conv_nn.parameters():
    print(parameter.shape)


CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
  )
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 16, 5, 5])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


### 4. Train the model on the dataset

In [ ]:
loss_train = []
loss_valid = []

def train(num_epochs, nn, loaders):
  min_valid_loss = np.inf

  if torch.cuda.is_available():
    nn.cuda()

  for epoch in range(num_epochs):

    # Train the model
    epoch_train_loss = 0
    # This line tells our NN that it's in the training mode
    # This will become relevant when we introduce layers that behave
    # differently in training and deployment/evaluation modes
    nn.train()
    for i, (images, labels) in enumerate(loaders['train']):
      if torch.cuda.is_available():
        images, labels = images.cuda(), labels.cuda()
      output = nn(images)
      loss = loss_f(output, labels)
      epoch_train_loss += loss.item()

      # clear gradients for this training step
      optimizer.zero_grad()

      # backpropagation, compute gradients
      loss.backward()
      # apply gradients
      optimizer.step()

    # Validate the model
    epoch_val_loss = 0
    nn.eval()
    for images_v, labels_v in loaders['validation']:
      if torch.cuda.is_available():
        images_v, labels_v = images_v.cuda(), labels_v.cuda()
      output = nn(images_v)
      loss_v = loss_f(output, labels_v)
      epoch_val_loss += loss_v.item()

    print(f'Epoch {epoch+1}')
    print(f'Training Loss: {epoch_train_loss / len(loaders["train"])}')
    print(f'Validation Loss: {epoch_val_loss / len(loaders["validation"])}')
    print('-------------------')
    if min_valid_loss > epoch_val_loss:
      print(f'Validation Loss Decreased({min_valid_loss}--->{epoch_val_loss}) \t Saving The Model')
      min_valid_loss = epoch_val_loss
      # Saving State Dict
      torch.save(nn.state_dict(), 'saved_model.pth')

train(20, conv_nn, loaders)

Epoch 1
Training Loss: 0.2574996976852417
Validation Loss: 0.31616960480809214
-------------------
Validation Loss Decreased(inf--->31.61696048080921) 	 Saving The Model
Epoch 2
Training Loss: 0.24570678855478764
Validation Loss: 0.30274931967258456
-------------------
Validation Loss Decreased(31.61696048080921--->30.274931967258453) 	 Saving The Model
Epoch 3
Training Loss: 0.23961457602679728
Validation Loss: 0.2943728061020374
-------------------
Validation Loss Decreased(30.274931967258453--->29.437280610203743) 	 Saving The Model
Epoch 4
Training Loss: 0.22976759906113148
Validation Loss: 0.30019139364361763
-------------------
Epoch 5
Training Loss: 0.22094300243258477
Validation Loss: 0.296290482878685
-------------------
Epoch 6
Training Loss: 0.21411211340129374
Validation Loss: 0.305352770537138
-------------------
Epoch 7
Training Loss: 0.20750077272951603
Validation Loss: 0.29769605785608294
-------------------
Epoch 8
Training Loss: 0.19912269833683968
Validation Loss: 0.

### 5. Plot the training curves (Loss and accuracy)

### 6. Show the confusion matrix and accuracy on the test dataset. (Don't do confussion matrix)

### 7. Is LeNet better than the fully connected Neural Network trained in the project from lecture 3? Comment on the results

Answer:

## Sub-project 2:

## Use a pretrained model to solve

In this project you will download the Intel Image Classification dataset (https://www.kaggle.com/puneet6060/intel-image-classification/download).
Find a suitable pretrained Convolutional Neural Network and its weights. Fix the filters and retrain/finetune the top of the network. Show and comment on resuls.

Alternativelly, you can browse Kaggle for some interesting datasets like: Dogs vs Cats, Alien vs Predator, Doom vs Animal Crossing, CelebA, etc. Just don't use something easy like MNIST, Fashion-MNIST, etc.

### 1. Download the data

### 2. Preprocess the data.

(This might include resizing, augmenting, etc.)




### 3. Split the data (train / test / validation) and make mini-batches.

### 4. Download the pretrained architecture and weights.

This part might include some research and understanding of architecures and the type of data they were trained on.

### 5. Build the larger model that includes the pre-trained part and prepare it for training.

* Show the model summary so you are sure which components are included and how many (un)trainable parameters you have

### 6. Train the trainable part of the model

### 7. Show the training plots and confusion matrix

### Finetune the model and show the plots, test data confusion matrix and accuracy.

### 9. Comment on the results. Are the results to your satisfaction? Which phase contributed to the improved performance? What was the most challenging aspect of the exercise? Ideas for improving the model?

Answer: